In [1]:
# Import packages
import numpy as np
import sys
import cv2
import os
import argparse
import ffmpeg
from tqdm import tqdm

In [2]:
directory = 'C:\\JA_test\\16_well'
min_pad = 5
min_well = 100
max_well = 150
examine_frame = 2000
target_num_well = 16

In [3]:
import numpy as np

def order_coordinates(num_wells,circles):
    well_order = np.arange(num_wells).reshape(int(np.sqrt(num_wells)),int(np.sqrt(num_wells)))
    row_idx = np.argsort(circles[0][:,0])
    sorted_row_coord = circles[0][row_idx]

    col_idx = []
    all_sorted_coord = np.zeros((num_wells,4))
    for i, value in enumerate(np.arange(0,np.sqrt(num_wells))):  # Loops thru each row (0-3)
        col_idx_temp = np.argsort(sorted_row_coord[well_order[i],1]) + i*np.sqrt(num_wells) # At that row, grab the x-values, and sorts them as an idx and adds row value 
        col_idx.append(col_idx_temp)
    col_idx = np.reshape(col_idx,(1,num_wells))[0]
    sorted_circles = sorted_row_coord[col_idx.astype(int)]

    return sorted_circles

In [ ]:
import numpy as np
import ffmpeg
import os

def run_segmentation(file_list,file_num,all_coord,num_wells):
    for well in np.arange(num_wells[file_num][0]):
        print(f'Running Well: {well+1}')
        try:
            stream = ffmpeg.input(file_list[file_num])
            stream = ffmpeg.filter(stream,'crop',\
                                   str(round(all_coord[file_num][well][0])),\
                                   str(round(all_coord[file_num][well][1])),\
                                   str(round(all_coord[file_num][well][2])),\
                                   str(round(all_coord[file_num][well][3])),\
                                  )
            
            target_dir = os.path.join(os.path.dirname(file_list[file_num]), 'well_videos')
            base_name = os.path.splitext(os.path.basename(file_list[file_num]))[0]
            target_file = os.path.join(target_dir,base_name + '_well_' + str(well+1) + '.avi')

            stream = ffmpeg.output(stream, target_file, **{'c:v': 'libx264rgb'},\
                                  minrate='4000k', maxrate='4000k', preset='medium', crf=10, f='avi')
            stream = ffmpeg.overwrite_output(stream)
            ffmpeg.run(stream,capture_stdout=True, capture_stderr=True)
        except ffmpeg.Error as e:
            print('stdout:', e.stdout.decode('utf8'))
            print('stderr:', e.stderr.decode('utf8'))
            raise e

In [3]:
# Import packages
import numpy as np
import os
import argparse
import multiprocessing
from ffmpeg_seg_comp import run_segmentation
from detect_and_export import detect_and_export

# Run through the folder and get all the avi names
file_list = [];
for filename in os.listdir(directory):
    if filename.endswith(".avi"): 
        file_list.append(os.path.join(directory, filename))
        continue
    else:
        continue

print(' ')
print('Total AVI Files: ' + str(np.shape(file_list)[0]))
print('###############################################')
print(' ')
print('Detecting Wells...')
num_wells,all_coord = detect_and_export(file_list,examine_frame,min_well,max_well,min_pad,target_num_well)
print(' ')
print('CHECK WELL-DETECTION OUTPUT TO COMFIRM')
print('###############################################')

# Segment Wells and export as avi
target_dir = os.path.join(os.path.dirname(file_list[0]), 'well_videos')
os.mkdir(target_dir)
jobs = []
for file_num in np.arange(np.shape(all_coord)[0]):
    p = multiprocessing.Process(target=run_segmentation, args=(file_list,file_num,all_coord,num_wells))
    jobs.append(p)
    p.start()

 
Total AVI Files: 3
###############################################
 
Detecting Wells...
File #1 ; Number of wells detected: 16
File #2 ; Number of wells detected: 16
File #3 ; Number of wells detected: 16
 
CHECK WELL-DETECTION OUTPUT TO COMFIRM
###############################################


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\JA_test\\16_well\\well_videos'

In [7]:
num_wells = []
all_coord =[]
for video in np.arange(np.shape(file_list)[0]):
    cap = cv2.VideoCapture(file_list[video])
    cap.set(1,examine_frame)
    ret, frame = cap.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=min_well,maxRadius=max_well)
    circles = np.uint16(np.around(circles))
    num_wells.append(np.shape(circles[0,:,0]))
    print('File #' + str(video+1) + ' ; Number of wells detected: ' + str(num_wells[video][0]))
    # sorted_circles = order_coordinates(target_num_well,circles)

    well_order = np.arange(target_num_well).reshape(int(np.sqrt(target_num_well)),int(np.sqrt(target_num_well)))
    row_idx = np.argsort(circles[0][:,0])
    sorted_row_coord = circles[0][row_idx]

    col_idx = []
    all_sorted_coord = np.zeros((target_num_well,4))
    for i, value in enumerate(np.arange(0,np.sqrt(target_num_well))):  # Loops thru each row (0-3)
        col_idx_temp = np.argsort(sorted_row_coord[well_order[i],1]) + i*np.sqrt(target_num_well) # At that row, grab the x-values, and sorts them as an idx and adds row value 
        col_idx.append(col_idx_temp)
    col_idx = np.reshape(col_idx,(1,target_num_well))[0]
    sorted_circles = sorted_row_coord[col_idx.astype(int)]

    for idx,i in enumerate(sorted_circles):
        cv2.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
        cv2.circle(frame,(i[0],i[1]),2,(0,0,255),3)
        cv2.putText(frame,'Well: ' +str(idx+1), (i[0]-40,i[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0))
    img_out_name = os.path.splitext(file_list[video])[0] + '_well_detection.jpg'
    cv2.imwrite(img_out_name, frame)
    if num_wells[video][0] > target_num_well:
        sys.exit("Too Many Wells detected! Check Pixel Range for well-detection!")
    if num_wells[video][0] < target_num_well:
        sys.exit("No Enough Wells detected! Check Pixel Range for well-detection!")
    coord_lines = np.zeros((num_wells[video][0],4))

    for well in range(num_wells[video][0]):
        coord_lines[well][0] = (max(sorted_circles[:,2])+min_pad)*2 # Width
        coord_lines[well][1] = (max(sorted_circles[:,2])+min_pad)*2 # Height
        coord_lines[well][2] = sorted_circles[well][0]-(max(sorted_circles[:,2])+min_pad) # Top left X
        coord_lines[well][3] = sorted_circles[well][1]-(max(sorted_circles[:,2])+min_pad) # Top left Y  
    all_coord.append(coord_lines)

File #1 ; Number of wells detected: 16


TypeError: unsupported operand type(s) for -: 'list' and 'int'